# ТЕМАТИЧЕСКОЕ МОДЕЛИРОВАНИЕ С ПОМОЬЮ LDA

In [50]:
import joblib
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import LdaMulticore

In [7]:
dialogues = joblib.load(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\dialogues_prepared.pkl')
intents = open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_intents.txt', encoding='utf-8').readlines()
intents =  [str(i).replace('\n', '') for i in intents]
phrases = open(r'F:\ОБУЧЕНИЕ\CHAT-BOTs\тексты\dialogues\_phrases.txt', encoding='utf-8').readlines()
phrases =  [str(i).replace('\n', '') for i in phrases]
kb = pd.read_excel(r'C:\Users\vndan\projects\netology_chat_bot\kb\kb_new.xlsx', sheet_name='base')

# объединим все диалогив один список
concatted_phrases = dialogues.participant_1.tolist() + dialogues.participant_2.tolist() + intents + phrases + kb['message'].tolist()
concatted_phrases[:2]

['Привет) расскажи о себе',
 'Что читаешь? Мне нравится классика . Я тоже люблю пообщаться']

In [10]:
# токенизируем, исключив лишние символы
def text2seq(phrase):
    return text_to_word_sequence(str(phrase), filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ')

text = [text2seq(phrase) for phrase in concatted_phrases]
text[:2]

[['привет', 'расскажи', 'о', 'себе'],
 ['что',
  'читаешь',
  'мне',
  'нравится',
  'классика',
  'я',
  'тоже',
  'люблю',
  'пообщаться']]

In [9]:
# обучим word2vec - на основе него будут созданы эмбеддинги
wvec = Word2Vec(sentences=text, size=350, window=5, min_count=3, seed=777, workers=-1, max_vocab_size=None)
# wvec.save('./models/word2vec.model')
# wvec = Word2Vec.load('./models/word2vec.model')

wvec.wv.__getitem__('как дела'.split(' ')).shape

(2, 350)

In [17]:
# создадим словарь слов в номер
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)

word2id = tokenizer.word_index
id2word = {v:k for k,v in word2id.items()}

sequences[:2]

[[12, 147, 31, 59], [17, 1407, 23, 79, 2501, 1, 22, 8, 136]]

In [19]:
from gensim import corpora

In [20]:
dct = corpora.Dictionary(text)
len(dct)

62412

In [21]:
corpus = [dct.doc2bow(phrase) for phrase in text]
corpus[:2]

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1)]]

In [33]:
################# LDA #########################
# попробуем выявить ключевые топики диалогов
lda_model = LdaMulticore(corpus=corpus,
                         id2word=dct,
                         random_state=777,
                         num_topics=300,
                         passes=10,
                         chunksize=1000,
                         batch=False,
                         alpha='asymmetric',
                         decay=0.5,
                         offset=64,
                         eta=None,
                         eval_every=0,
                         iterations=100,
                         gamma_threshold=0.001,
                         minimum_probability = 0.05,
                         per_word_topics=True,
                         dtype=np.float32)
# save the model
lda_model.save('lda_model_chatbot.model')
# See the topics
lda_model.print_topics(-1)

[(0,
  '0.291*"представляю" + 0.054*"🙃" + 0.016*"меняется" + 0.014*"стимул" + 0.014*"идите" + 0.009*"😊😂" + 0.007*"нынешней" + 0.007*"полноценная" + 0.007*"системой" + 0.007*"ужс"'),
 (1,
  '0.145*"у" + 0.089*"меня" + 0.072*"есть" + 0.063*"тебя" + 0.052*"а" + 0.022*"вас" + 0.019*"и" + 0.018*"нет" + 0.014*"я" + 0.011*"хобби"'),
 (2,
  '0.546*"мне" + 0.259*"тебе" + 0.161*"нравится" + 0.006*"хотите" + 0.002*"пойдёт" + 0.001*"кожи" + 0.001*"расслабляться" + 0.001*"пробуй" + 0.001*"злые" + 0.001*"сальсу"'),
 (3,
  '0.541*"мы" + 0.104*"друг" + 0.078*"оо" + 0.034*"с" + 0.032*"ходим" + 0.014*"парк" + 0.012*"другу" + 0.012*"оставаться" + 0.012*"котами" + 0.011*"лошади"'),
 (4,
  '0.149*"привет" + 0.100*"как" + 0.096*"чем" + 0.067*"ты" + 0.046*"а" + 0.044*"занимаешься" + 0.037*"дела" + 0.035*"работаешь" + 0.033*"зовут" + 0.028*"кем"'),
 (5,
  '0.086*"пока" + 0.065*"приятно" + 0.051*"было" + 0.040*"ну" + 0.039*"да" + 0.036*"ладно" + 0.030*"пора" + 0.026*"очень" + 0.025*"мне" + 0.025*"вам"'),
 (6,


In [ ]:
# Cложность (Perplexity) модели
print('Perplexity: ', lda_model.log_perplexity(corpus))

In [ ]:
# согласованность (Coherence) темы
# в идеале построить несколько моделей LDA с разным количество топиков, выбрать с большим знаением когерентности
from gensim.models import CoherenceModel

coherence_model_lda = CoherenceModel(model=lda_model, texts=text, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

In [55]:
topics_dict = {k:v for k,v in enumerate(lda_model.print_topics(-1))}
pd.DataFrame(topics_dict).to_excel(r'C:\Users\vndan\projects\netology_chat_bot\kb\themes_LDA.xlsx')

In [34]:
theme_matrix=lda_model.get_topics()

In [37]:
def get_relevant_themes(text, theme_matrix, token2id):
    
    scores = np.zeros(len(theme_matrix))
    for ew in text.split():
        if ew in token2id:
            for i in range(len(theme_matrix)):
                scores[i] += theme_matrix[i, token2id[ew]]
        else:
            print(f'unknown word {ew}')
    
    top = np.argsort(scores)[::-1][:4]   
    delta = scores[top][:3] - scores[top][1:]
    result_themes = [top[0]]
    result_scores = [scores[top[0]]]
    for i in range(3):
        if delta[i] > 0.1 or top[i+1]<0.1: # странно - будет работать только если дельта равна 0,1, иначе пропускает        
            break
        else:
            result_themes.append(top[i+1])
            result_scores.append(scores[top[i+1]])
    return result_themes, result_scores

In [38]:
corpus_themes = [get_relevant_themes(' '.join(phrase), theme_matrix, dct.token2id) for phrase in text]

In [39]:
%%time
theme_stats = [[] for _ in range(len(theme_matrix))]
for i, (th, _) in enumerate(corpus_themes):
    for ix in th:
        theme_stats[ix].append(i)

Wall time: 99.3 ms


In [40]:
themes2sentence=pd.DataFrame(columns=['Номер_темы'], index=range(len(text)))
for i in range(len(theme_stats)):
    for sentence in theme_stats[i]:
        themes2sentence.iloc[sentence,:]=i

In [41]:
themes2sentence.shape[0]==len(text)

True

In [42]:
themes2sentence['original_text'] = concatted_phrases
themes2sentence['preprocessed_text'] = text

In [53]:
themes2sentence.to_excel(r'C:\Users\vndan\projects\netology_chat_bot\kb\topics_LDA.xlsx')

можно также попробовать LSI

In [ ]:
from gensim.models import LsiModel
# Build the LSI Model
lsi_model = LsiModel(corpus=corpus, id2word=dct, num_topics=2, decay=0.5)
# View Topics
print(lsi_model.print_topics(-1))